In [ ]:
pip install langchain-community llama-cpp-python pydantic==1.10.8 email extract-msg PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 9.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
pip install langchain-community

In [ ]:
pip install llama-cpp-python

In [ ]:
pip install pydantic==2.7.4

In [ ]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
pip install email

In [ ]:
# filepath: c:\Users\ajosh\OneDrive\Desktop\MachineLearning\Hackathon2025\loan_classifier.py
from langchain_community.llms import LlamaCpp
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from pathlib import Path
import json
import PyPDF2
from google.colab import files
import io

class ServiceRequest(BaseModel):
    request_type: str = Field(description="Main category of the request")
    sub_request: str = Field(description="Specific sub-category of the request")
    confidence_score: float = Field(description="Confidence score of the classification")
    supporting_text: str = Field(description="Relevant text that supports this classification")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install gpt4all


In [ ]:
from gpt4all import GPT4All
model = GPT4All("Meta-Llama-3-8B-Instruct.Q4_0.gguf") # downloads / loads a 4.66GB LLM
with model.chat_session():
    print(model.generate("How can I run LLMs efficiently on my laptop?", max_tokens=1024))

In [ ]:
!pip install faiss-cpu sentence-transformers chromadb

In [ ]:
#pip install .embedder

ERROR: Invalid requirement: '.embedder': Expected package name at the start of dependency specifier
    .embedder
    ^


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from typing import List
import os

class DocumentEmbedder:
    def __init__(self):
        self.embeddings = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'}
        )
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50,
            separators=["\n\n", "\n", " ", ""]
        )

    def create_embeddings(self, email_content: str, attachment_content: str) -> FAISS:
        # Combine content with metadata
        documents = []
        if email_content:
            documents.append(Document(
                page_content=email_content,
                metadata={"source": "email"}
            ))
        if attachment_content:
            documents.append(Document(
                page_content=attachment_content,
                metadata={"source": "attachment"}
            ))

        # Split documents
        split_docs = self.text_splitter.split_documents(documents)

        # Create and persist vector store
        vector_store = FAISS.from_documents(split_docs, self.embeddings)

        # Save embeddings locally
        vector_store.save_local("document_embeddings")

        return vector_store

    def query_similar_content(self, query: str, vector_store: FAISS, k: int = 3) -> List[Document]:
        return vector_store.similarity_search(query, k=k)

In [ ]:
def read_pdf_content(pdf_file):
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text() + "\n"
        return text
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

def process_uploaded_files():
    print("Upload your email file (EML/MSG) and any attachments...")
    uploaded = files.upload()

    email_content = ""
    attachment_content = ""

    for filename, content in uploaded.items():
        if filename.lower().endswith(('.eml', '.msg')):
            email_content = content.decode('utf-8')
        elif filename.lower().endswith('.pdf'):
            attachment_content = read_pdf_content(io.BytesIO(content))

    return email_content, attachment_content

In [ ]:
# class LoanServiceRequestClassifier:
#     def __init__(self, model_path: str = "llama-3-7b.gguf"):
#         self.llm = LlamaCpp(
#             model_path=model_path,
#             temperature=0.1,
#             max_tokens=2000,
#             n_gpu_layers=40,
#             n_ctx=4096,
#             top_p=0.95,
#             verbose=True
#         )

#from langchain_community.llms import HuggingFaceHub
#import os

from langchain_community.llms import GPT4All
#from .embedder import DocumentEmbedder


# class LoanServiceRequestClassifier:
#     def __init__(self):
#         # Set your Hugging Face API token
#         os.environ["HUGGINGFACEHUB_API_TOKEN"] = "xXXXxXXX"

#         self.llm = HuggingFaceHub(
#             repo_id="meta-llama/Llama-3.2-3B-Instruct",  # Replace with actual Llama 3 model ID
#             model_kwargs={
#                 "temperature": 0.1,
#                 "max_tokens": 2000,
#                 "top_p": 0.95
#             }
#         )

class LoanServiceRequestClassifier:
    def __init__(self):
        self.llm = GPT4All(

            #model="gpt4all-falcon-q4_0.gguf",
            model="Meta-Llama-3-8B-Instruct.Q4_0.gguf",
            n_threads=8,
            verbose=True
        )
        self.embedder = DocumentEmbedder()

        self.output_parser = PydanticOutputParser(pydantic_object=ServiceRequest)
        self._setup_prompt()


    def _setup_prompt(self):
        self.prompt = ChatPromptTemplate.from_messages([
            ("system ", """You are a loan servicing expert. Analyze the provided text and classify it into appropriate request and sub-request categories.

            Request Types and Sub-categories:
            1. Loan Modification
               - Payment Reduction
               - Term Extension
               - Interest Rate Adjustment
               - Principal Forbearance
               - Hardship Assistance

            2. Payment Processing
               - Payment Setup/Change
               - Auto-Pay Enrollment
               - Payment Dispute
               - Late Payment Resolution
               - Payment Application

            3. Documentation
               - Statement Request
               - Payment History
               - Tax Documents (1098)
               - Loan Agreement Copy
               - Insurance Documents

            4. Account Updates
               - Contact Information
               - Insurance Updates
               - Banking Details
               - Name/Ownership Change
               - Death/Estate Processing

            5. Payoff/Discharge
               - Payoff Statement
               - Early Payoff Quote
               - Lien Release
               - Satisfaction Letter
               - Discharge Processing

            Analyze the content and output a JSON object with:
            - request_type: Main category
            - sub_request: Specific sub-category
            - confidence_score: 0.0 to 1.0
            - supporting_text: Relevant evidence"""),
            ("human", "{content}")
        ])

    def classify_request(self, email_content: str, attachment_text: str = "") -> ServiceRequest:
        # full_content = f"Email:\n{email_content}\n\nAttachment:\n{attachment_text}"
        # chain = self.prompt | self.llm | self.output_parser
        # return chain.invoke({"content": full_content})
        # Create embeddings
        vector_store = self.embedder.create_embeddings(email_content, attachment_text)

        # Get most relevant chunks for classification
        relevant_chunks = self.embedder.query_similar_content(
            "loan service request classification",
            vector_store
        )

        # Prepare content for classification
        processed_content = "\n".join([
            f"Content from {chunk.metadata['source']}:\n{chunk.page_content}\n"
            for chunk in relevant_chunks
        ])

        # Classify using LLM
        chain = self.prompt | self.llm | self.output_parser
        return chain.invoke({"content": processed_content})

In [ ]:
def main():
    # Download Llama 3 model (you'll need to modify this based on your setup)
   # MODEL_PATH = "llama-7b.Q5_K_M.gguf"

    # Process uploaded files
    email_content, attachment_content = process_uploaded_files()

    # Initialize classifier
    #classifier = LoanServiceRequestClassifier(model_path=MODEL_PATH)
    classifier = LoanServiceRequestClassifier()

    # Get classification
    try:
        classification = classifier.classify_request(
            email_content=email_content,
            attachment_text=attachment_content
        )

        print("\nClassification Results:")
        print(f"Request Type: {classification.request_type}")
        print(f"Sub-Request: {classification.sub_request}")
        print(f"Confidence Score: {classification.confidence_score:.2f}")
        print(f"\nSupporting Text:\n{classification.supporting_text}")

        # Save results as JSON
        # results = classification.dict()
        results = {
            "classification": classification.dict(),
            "embeddings_path": "document_embeddings"
        }
        with open('classification_results.json', 'w') as f:
            json.dump(results, f, indent=2)

        # Download results
        files.download('classification_results.json')

    except Exception as e:
        print(f"Error during classification: {str(e)}")

if __name__ == "__main__":
    main()

Upload your email file (EML/MSG) and any attachments...


Saving cfpb_mortgage-servicing-h-4d4_sample-form-updated.pdf to cfpb_mortgage-servicing-h-4d4_sample-form-updated (1).pdf


<ipython-input-8-df79ee42a4a8>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]